<a href="https://colab.research.google.com/github/levi3001/cs576_assigment/blob/main/assignment2_2021S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS576 Assignment #2: Image Classification using Convolutional Neural Networks (CNNs) 
---
Primary TA : Whie Jung

TA's E-mail : whieya@kaist.ac.kr, wogns98@kaist.ac.kr 

QnA Channel: Same Slack workspace but with *assignment2* channel (https://join.slack.com/t/kaistcs576/shared_invite/zt-o3gqak0y-yj3NCb_SQFxVkqO0U6PWYw)

---

## Instruction
- In this assignment, we will classify the images in CIFAR10 dataset into 10 categories (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck) using Convolutional Neural Networks.  

- For this, you first need to implement necessary network components (e.g. residual blocks) using nn.Module class. Then, you need to implement data pipeline. Finally, you need to implement an entire training/test pipeline.  

- In each part, you will be given a starter code for the implementation. Please read the attached illustrations and instructions carefully to implement the codes.  

- As you follow the given steps, fill in the section marked ***Px.x*** (e.g. P1.1, P1.2, etc) with the appropriate code.

- DO NOT modify any of the skeleton codes except the area where we allow you to change. Please write your codes only in the designated area.

## Submission guidelines
- Your code and report will be all in Colab. 
- Go to the [link](https://drive.google.com/drive/folders/1XQT-Mk31_jc3KqXYX1FyMIcmtLutrKzj?usp=sharing), find `assignment2.ipynb` and `dataset.tar.gz`, save them into your own google drive by clicking `make a copy(사본만들기)`. Find the copies in your drive, change their name to `assignment2.ipynb` and `dataset.tar.gz`, respectively, if their names were changed to e.g. `Copy of assignment2.ipyb` or `assignment2.ipynb의 사본`. Also, keep them in a single directory.
- <font color="red"> You will get the full credit **only if** you complete the code **and** write a discussion of the results in the discussion section at the bottom of this page. </font>
- We should be able to reproduce your results using your code. Please double-check if your code runs without error and reproduces your results. Submissions failed to run or reproduce the results will get a substantial penalty. 

## Deliverables
- Download your Colab notebook and submit it in a format as : **[StudentID].ipynb**.
- Your assignment should be submitted through KLMS. All other submissions (e.g., via email) will not be considered as valid submissions. 

## Due date
- **23:59:59 May 4th.**
- Late submission is allowed until 23:59:59 May 6th.
- Late submission will be applied 20% penalty.


## Questions
- We will use the same SLACK channel (https://join.slack.com/t/kaistcs576/shared_invite/zt-o3gqak0y-yj3NCb_SQFxVkqO0U6PWYw) as a main communication channel. When you post questions, please make it public so that all students can share the information. 


## Changelog
- 04/18 03:02 : Added the cells defining result_dir and log_dir above the cell executing the tensorboard. Also added L12~L17 in the function train().
- 04/18 14:01 : Removed remounting (the part including drive.mount('/content/drive'))

---

---


# Prerequisite: change the runtime type to **GPU**.

![test](https://docs.google.com/uc?export=download&id=1Jugrjl86L9EY1ePTjH8OVMFq7gmZsoz_)

---
# Prerequisite: mount your gdrive.

In [ ]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


---
# Prerequisite: setup the `root` directory properly.

In [ ]:
# Specify the directory path where `assignemnt2.ipynb` and `dataset.tar.gz` exist.
# For example, if you saved `assignment2.ipynb` and `dataset.tar.gz` in `/gdrive/My Drive/cs576/assignment2` directory,
# then set root = '/gdrive/My Drive/cs576/assignment2'
root = '/directory/path/where/assignment2.ipynb/exists'
root = '/gdrive/My Drive/Colab Notebooks/Copy of assignment2'

# Extract copied `dataset.tar.gz`.
!tar -xzf '/gdrive/My Drive/'{root.replace('/gdrive/My Drive/', '').strip('/')}'/dataset.tar.gz'

tar (child): /gdrive/My Drive/Colab: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


---
# Import libraries

In [ ]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

-----

# Network Modules

In this section, you need to implement three modularized layer (or network) classes as follows:

(1) plain residual block (ResBlockPlain)  
(2) residual block with bottleneck (ResBlockBottleneck)  
(3) an entire network module (MyNetwork)  


In each cell, there is a starter code as well as a schematic illustration and instruction for implementing that module class. Specifically, the schematic illustrations are to show you the computational graphs of modules, which give you high-level views on how the modules should be constructed and work. (E.g. which nn.Module to use, or input/output shape of each layer written in italics). Therefore, please read the illustrations and instructions carefully to complete the codes.

Below is an example.

### Example: ConvLayer Module [(Illustration)](https://docs.google.com/drawings/d/1_aPhPSPgh5-5FEfI_jnfp8r6-wNjY_QYXBT3zzjkHk0/edit?usp=sharing)

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, activation_type='relu', use_bn=False):
        super(ConvLayer, self).__init__()
        """Initialize a basic convolutional layer module components.

        Illustration: https://docs.google.com/drawings/d/1_aPhPSPgh5-5FEfI_jnfp8r6-wNjY_QYXBT3zzjkHk0/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link. 
            2. Initialized network components will be referred in `forward` method 
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input. 
            2. out_channels (int): Number of channels produced by the convolution.
            3. activation_type (string, optional): Type of non-linear activation function to use. (default: 'relu') 
            4. use_bn (bool, optional): Whether to use batch normalization. (default: False)
        """
        ##########################
        ## Write your code here ##
        self.conv = nn.Conv2d(in_channels, out_channels, 3, 1, 1)
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity() 

        if activation_type == 'relu':
            self.act = nn.ReLU(True)
        elif activation_type == 'lrelu':
            self.act = nn.LeakyReLU(0.2, True)
        elif activation_type == 'sigmoid':
            self.act = nn.Sigmoid()
        elif activation_type == 'tanh':
            self.act = nn.Tanh()
        elif activation_type == 'none':
            self.act = nn.Identity() 
        else:
            raise ValueError('Unknown activation_type !')
        ##########################

    def forward(self, x):
        """Feed-forward the data `x` through the module.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link 
               using the initialized components in the __init__ method.

        Args:
            1. x (torch.FloatTensor): A tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels, H, W). 

        """
        ###########################
        ## Write your code here ##
        output = self.conv(x)
        output = self.bn(output)
        output = self.act(output)
        ###########################
        return output

In [ ]:
# Check and test your ConvLayer here
# You may modify this cell for debugging

in_channels = 8
out_channels = 16
activation_type = 'relu'
use_bn = True

convlayer_test = ConvLayer(in_channels, out_channels, activation_type, use_bn)
print(convlayer_test)

B, C, H, W = 1, in_channels, 32, 32
x_test = torch.randn(1, C, H, W)
print('input shape: ', x_test.shape, '| dtype: ', x_test.dtype)

output = convlayer_test(x_test)
print('output shape: ', output.shape, '| dtype: ', output.dtype)


### 1. Implement ResBlockPlain [(Illustration)](https://docs.google.com/drawings/d/1N0vi9S-RwDAjyJoC9eCVWwHnlKXfSlflf2xWTGEFRFQ/edit?usp=sharing) (10pt)

In [ ]:
class ResBlockPlain(nn.Module):
    def __init__(self, in_channels, use_bn=False):
        super(ResBlockPlain, self).__init__()
        """Initialize a residual block module components.

        Illustration: https://docs.google.com/drawings/d/1N0vi9S-RwDAjyJoC9eCVWwHnlKXfSlflf2xWTGEFRFQ/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link. 
            2. Initialized network components will be referred in `forward` method 
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input.
            2. use_bn (bool, optional): Whether to use batch normalization. (default: False)
        """
        #################################
        ## P1.1. Write your code here ##
        #self.dummy_conv = nn.Conv2d(1, 1, 1, 1, 0) # Note: you must erase this line
        self.conv=self.Conv2d(in)
        #################################

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link 
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): An tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels, H, W). 
        """
        ################################
        ## P1.2. Write your code here ##
        output = dummy_output = x # Note: you must erase this line
        ################################
        return output 

In [ ]:
# Check and test your ResBlockPlain here
# You may modify this cell for debugging

in_channels = 16
use_bn = True

resblockplain_test = ResBlockPlain(in_channels, use_bn)
print(resblockplain_test)

B, C, H, W = 1, in_channels, 32, 32
x_test = torch.randn(1, C, H, W)
print('input shape: ', x_test.shape, '| dtype: ', x_test.dtype)

output = resblockplain_test(x_test)
print('output shape: ', output.shape, '| dtype: ', output.dtype)

### 2. Implement ResBlockBottleneck [(Illustration)](https://docs.google.com/drawings/d/1cpqMoRKtVvLy6Zwt7HziEm3DyGsbNF6jYCTCCbm5WZY/edit?usp=sharing) (10pt)

In [ ]:
class ResBlockBottleneck(nn.Module):
    def __init__(self, in_channels, hidden_channels, use_bn=False):
        super(ResBlockBottleneck, self).__init__()
        """Initialize a residual block module components.

        Illustration: https://docs.google.com/drawings/d/1cpqMoRKtVvLy6Zwt7HziEm3DyGsbNF6jYCTCCbm5WZY/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link. 
            2. Initialized network components will be referred in `forward` method 
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input. 
            2. hidden_channels (int): Number of hidden channels produced by the first ConvLayer module.
            3. use_bn (bool, optional): Whether to use batch normalization. (default: False)
        """
        #################################
        ## P2.1. Write your code here ##
        self.dummy_conv = nn.Conv2d(1, 1, 1, 1, 0) # Note: you must erase this line
        #################################

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link 
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): An tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels, H, W). 
        """
        ################################
        ## P2.2. Write your code here ##
        output = dummy_output = x # Note: you must erase this line
        ################################
        return output 

In [ ]:
# Check and test your ResBlockBottleneck here
# You may modify this cell for debugging

in_channels = 16
hidden_channels = 8
use_bn = True

resblockbottleneck_test = ResBlockBottleneck(in_channels, hidden_channels, use_bn)
print(resblockbottleneck_test)

B, C, H, W = 1, in_channels, 32, 32
x_test = torch.randn(1, C, H, W)
print('input shape: ', x_test.shape, '| dtype: ', x_test.dtype)

output = resblockbottleneck_test(x_test)
print('output shape: ', output.shape, '| dtype: ', output.dtype)

### 3. Implement MyNetwork [(Illustration)](https://docs.google.com/drawings/d/1dN2RLaCpK5W61A9s2WhdOfZDuDBn6JtIJmWmIAIMgtg/edit?usp=sharing) (20pt)

In [ ]:
class MyNetwork(nn.Module):
    def __init__(self, nf, resblock_type='plain', num_resblocks=[1, 1, 1], use_bn=False):
        super(MyNetwork, self).__init__()
        """Initialize an entire network module components.

        Illustration: https://docs.google.com/drawings/d/1dN2RLaCpK5W61A9s2WhdOfZDuDBn6JtIJmWmIAIMgtg/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link. 
            2. Initialized network components will be referred in `forward` method 
               for constructing the dynamic computational graph.

        Args:
            1. nf (int): Number of output channels for the first nn.Conv2d Module. An abbreviation for num_filter.
            2. resblock_type (str, optional): Type of ResBlocks to use. ('plain' | 'bottleneck'. default: 'plain')
            3. num_resblocks (list or tuple, optional): A list or tuple of length 3. 
               Each item at i-th index indicates the number of residual blocks at i-th Residual Layer.  
               (default: [1, 1, 1])
            4. use_bn (bool, optional): Whether to use batch normalization. (default: False)
        """
        ################################
        ## P3.1. Write your code here ##
        self.dummy_conv = nn.Conv2d(1, 1, 1, 1, 0) # Note: you must erase this line
        ################################

        # When all components are initialized, perform weight initialization on weights and biases.
        self.apply(self.init_params)

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link 
               using the initialized network components in __init__ method.
        Args:
            1. x (torch.FloatTensor): An image tensor of shape (B, 3, 32, 32).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, 10). 
        """
        ################################
        ## P3.2. Write your code here ##
        output = torch.tensor([[-0.1, 0.2, -0.3, 0.4, -0.5, 0.6, -0.7, 0.8, -0.9, 1.0]]).float() # Note: you must erase this line
        ################################
        return output

    def init_params(self, m):
        """Perform weight initialization on model parameters.

        Instructions:
            1. For nn.Conv2d and nn.Linear modules, 
               initialize their weights using Kaiming He Normal initialization,
               and initialize their biases with zeros.

            2. For nn.BatchNorm2d modules,
               initialize their weights with ones,
               and initizlie their biases with zeros.

            3. Otherwise, do not perform initialization.

            4. No need to return anything in this method.

            5. Hint: refer to the page 44 of the 'lecture note: tutorial on Pytorch [04/12]'

        Args:
            1. m (nn.Module) 
        """
        ################################
        ## P3.3. Write your code here ##
        ################################

    def compute_loss(self, logit, y):
        """Compute cross entropy loss.

        Hint: 
            If logit = torch.tensor([[-0.1, 0.2, -0.3, 0.4, -0.5, 0.6, -0.7, 0.8, -0.9, 1.0]]).float(),
            and y = torch.ones(1).long(), then loss value equals to 2.3364xxxx

        Args:
            1. logit (torch.FloatTensor): A tensor of shape (B, 10). 
            2. y (torch.LongTensor): A tensor of shape (B).

        Returns:
            1. loss (torch.FloatTensor): Computed cross entropy loss.
        """
        ################################
        ## P3.4. Write your code here ##
        loss = torch.tensor(2.3364) # Note: you must erase this line
        ################################
        return loss

In [ ]:
# Check and test your Network here
# You may modify this cell for debugging

num_filters = 16
num_resblocks = [1, 1, 1]
resblock_type = 'bottleneck'
use_bn = True

mynetwork_test = MyNetwork(num_filters, resblock_type, num_resblocks, use_bn)
print(mynetwork_test)

B, C, H, W = 1, 3, 32, 32
x_test = torch.randn(1, C, H, W)
y_test = torch.ones(1).long()
print('input shape: ', x_test.shape, '| dtype: ', x_test.dtype)
print('label shape: ', y_test.shape, '| dtype: ', y_test.dtype)

logit = mynetwork_test(x_test)
print('logit shape: ', logit.shape, '| dtype: ', logit.dtype)

loss_test = mynetwork_test.compute_loss(logit, y_test)
print('computed loss:', loss_test.item())

---

# Dataset and DataLoader

In this section, you need to implement data pipeline, as illustrated in **lecture note: Pytorch tutorial [4/12]**

Like Network section, you are provided with starter codes for the data pipeline.

Please refer to the instructions carefully to complete the codes.

### 4-1. Implement CIFAR10 Dataset Class (15pt)

In [ ]:
class CIFAR10(Dataset):
    """Customized `CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.

    Read the following descriptions on the dataset directory structure carefully to implement this `CIFAR10` class.

    In `dataset/cifar10` directory, you have `train` and `test` directories,
    each of which contains CIFAR10 images for the train and test, respectively.

    Also, there are 10 sub-directories (from `0` to `9`) in `train` and `test` directories, 
    where the name of each sub-directory is specified by CIFAR10 classes and 
    each sub-directory contains images for those classes. 

    For train data, there are 10*4,800=48,000 images in total (4,800 images for each class), 
    whereas test data consists of 10*1,200=12,000 images (1,200 images for each class). 

    For example,

    datset
        `-- cifar10
            |-- train
                |-- 0
                    |-- 00001.png
                    |-- ...
                    `-- 04800.png
                |-- ...
                `-- 9
                    |-- 00001.png
                    |-- ...
                    `-- 04800.png
            `-- test
                |-- 0
                    |-- 04801.png
                    |-- ...
                    `-- 06000.png
                |-- ...
                `-- 9
                    |-- 04801.png
                    |-- ...
                    `-- 06000.png

    """
    def __init__(self, root, train=True, transform=None):
        super(CIFAR10, self).__init__()
        """
        Instructions: 
            1. Assume that `root` equals to `dataset/cifar10`.

            2. If `train` is True, then parse all paths of train images, and keep them in the list `self.paths`. 
               E.g.) self.paths = ['dataset/cifar10/train/0/00001.png', ..., 'dataset/cifar10/train/9/4800.png']
               Also, the length of `self.paths` list should be 48,000.
                    
            3. If `train` is False, then parse all paths of test images, and keep them in the list `self.paths`. 
               E.g.) self.paths = ['dataset/cifar10/test/0/04801.png', ..., 'dataset/cifar10/test/9/06000.png']
               Also, the length of `self.paths` list should be 12,000.

        Args:
            root (string): Root directory of dataset where directory ``cifar10`` exists.
            train (bool, optional): If True, creates dataset from training set, otherwise
                creates from test set. (default: True)
            transform (callable, optional): A function/transform that takes in an PIL image
                and returns a transformed version. E.g, ``transforms.RandomCrop`` (default: None)
        """
        self.transform = transform 

        ################################
        ## P4.1. Write your code here ##
        num_paths = 48000 if train is True else 12000                                 # Note: you must erase this line
        dummy_paths = ['dataset/cifar10/train/0/00001.png' for _ in range(num_paths)] # Note: you must erase this line
        self.paths = dummy_paths                                                      # Note: you must erase this line 
        ################################

        assert isinstance(self.paths, (list,)), 'Wrong type. self.paths should be list.'
        if train is True:
            assert len(self.paths) == 48000, 'There are 48,000 train images, but you have gathered %d image paths' % len(self.paths)
        else:
            assert len(self.paths) == 12000, 'There are 12,000 test images, but you have gathered %d image paths' % len(self.paths)

    def __getitem__(self, idx):
        """
        Instructions:
            1. Given a path of an image, which is grabbed by self.paths[idx], infer the class label of the image.
            2. Convert the inferred class label into torch.LongTensor with shape (), and keep it in `label` variable.` 

        Args:
            idx (int): Index of self.paths

        Returns:
            image (torch.FloatTensor): An image tensor of shape (3, 32, 32).
            label (torch.LongTensor): A label tensor of shape ().
        """

        path = self.paths[idx] 
        # P4.2. Infer class label from `path`,
        # write your code here.

        # P4.3. Convert it to torch.LongTensor with shape ().
        # label = write_your_code_here (one-liner).
        label = torch.tensor(0) # Note: you must erase this line

        image = Image.open(path)
        if self.transform is not None:
            image = self.transform(image) 

        return image, label

    def __len__(self):
        return len(self.paths)

In [ ]:
# Check and test your CIFAR10 Dataset class here.
# You may modify this cell for debugging

data_dir = 'dataset/cifar10'
train = True
transform = transforms.ToTensor()

dset = CIFAR10(data_dir, train, transform)
print('num data:', len(dset))

x_test, y_test = dset[0]
print('image shape:', x_test.shape, '| type:', x_test.dtype)
print('label shape:', y_test.shape, '| type:', y_test.dtype)

### 4-2. Implement DataLoader (5pt)

In [ ]:
def get_dataloader(args):
    transform = transforms.Compose([
        transforms.ToTensor(),
        ])
    train_dataset = CIFAR10(args.dataroot, train=True, transform=transform)
    test_dataset = CIFAR10(args.dataroot, train=False, transform=transform)

    # P4.4. Use `DataLoader` module for mini-batching train and test datasets.
    # train_dataloader = DataLoader(WRITE_YOUR_CODE_HERE, batch_size=args.batch_size, shuffle=True, drop_last=True)
    # test_dataloader = DataLoader(WRITE_YOUR_CODE_HERE, batch_size=args.batch_size, shuffle=False, drop_last=False)

    return train_dataloader, test_dataloader

---

# 5. Train/Test Pipeline (40pt)

In this section, you need to implement the entire train and test loop in the pipeline.

Specifically, you need to do the followings:
1. feed inputs into the network, get outputs, and then compute classification loss. 
2. backward the computed loss and update network weights (only in the training loop).
3. save tensorboard logs frequently.
4. save checkpoint weights frequently.

Please refer to the **lecture note: Pytorch Tutorial [4/12]**. There are a lot of hints for implementing this pipeline !

In [ ]:
# Configurations & Hyper-parameters
# You may modify this cell for your experiments. 

from easydict import EasyDict as edict

args = edict()

# basic options 
args.name = 'main'                   # experiment name.
args.resume = False                  # whether to resume. If you want to resume training, change this option.
args.ckpt_dir = 'ckpts'              # checkpoint directory name.
args.ckpt_reload = '10'              # If you want to resume training, specify which epoch's checkpoint to re-load.
args.gpu = True                      # whether or not to use gpu. 

# network options
args.num_filters = 32                # number of output channels in the first nn.Conv2d module in MyNetwork.
args.resblock_type = 'bottleneck'    # type of residual block. ('plain' | 'bottleneck').
args.num_resblocks = [1, 2, 3]       # number of residual blocks in each Residual Layer.
args.use_bn = False                  # whether or not to use batch normalization.

# data options
args.dataroot = 'dataset/cifar10'    # where CIFAR10 images exist.
args.batch_size = 64                 # number of mini-batch size.

# training options
args.lr = 0.0001                     # learning rate.
args.epoch = 50                      # training epoch.

# tensorboard options
args.tensorboard = True             # whether or not to use tensorboard logging.
args.log_dir = 'logs'                # to which tensorboard logs will be saved.
args.log_iter = 100                  # how frequently logs are saved.

In [ ]:
# Added 04/18
# Basic settings
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'

result_dir = Path(root) / 'results' /args.name
ckpt_dir = result_dir / args.ckpt_dir
ckpt_dir.mkdir(parents=True, exist_ok=True)
log_dir = result_dir / args.log_dir
log_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Setup tensorboard.
if args.tensorboard:
    from torch.utils.tensorboard import SummaryWriter 
    writer = SummaryWriter(log_dir)
    %load_ext tensorboard
    %tensorboard --logdir '/gdrive/My Drive/'{str(log_dir).replace('/gdrive/My Drive/', '')}
else:
    writer = None

In [ ]:
def train(args):

    # Basic settings
    device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'

    result_dir = Path(root) / 'results' /args.name
    ckpt_dir = result_dir / args.ckpt_dir
    ckpt_dir.mkdir(parents=True, exist_ok=True)
    log_dir = result_dir / args.log_dir
    log_dir.mkdir(parents=True, exist_ok=True)

    # Added 04/18
    if args.tensorboard:
        from torch.utils.tensorboard import SummaryWriter 
        writer = SummaryWriter(log_dir)
    else:
        writer = None
        
    epoch = 0
    global_step = 0
    best_accuracy = 0.

    # Define your model and optimizer
    # Complete ResBlockPlain, ResBlockBottleneck, and MyNetwork modules to proceed further.
    net = MyNetwork(args.num_filters, args.resblock_type, args.num_resblocks, args.use_bn).to(device)
    optimizer = optim.Adam(net.parameters(), lr=args.lr)

    # Resume the training 
    if args.resume:
        ckpt_path = ckpt_dir / ('%s.pt' % args.ckpt_reload)

        try:
            checkpoint = torch.load(ckpt_path)
            net.load_state_dict(checkpoint['net'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            epoch = checkpoint['epoch'] + 1
            best_accuracy = checkpoint['best_accuracy']
            print(f'>> Resume training from epoch {epoch+1}')

        except Exception as e:
            print(e)

    # Get train/test data loaders  
    # Complete CIFAR10 dataset class and get_dataloader method to proceed further.
    train_dataloader, test_dataloader = get_dataloader(args)

    # Start training
    # Save the starting time
    start_time = time.time()

    for epoch in range(epoch, args.epoch):
        # start time
        _start_time = time.time() 

        # Here starts the train loop.
        net.train()
        for x, y in train_dataloader:
            global_step += 1

            # P5.1. Send `x` and `y` to either cpu or gpu using `device` variable. 
            # x = write your code here (one-liner). 
            # y = write your code here (one-liner).
            
            # P5.2. Feed `x` into the network, get an output, and keep it in a variable called `logit`. 
            # logit = write your code here (one-liner).

            # P5.3. Compute loss using `logit` and `y`, and keep it in a variable called `loss` 
            # loss =  write your code here (one-liner).
            accuracy = (logit.argmax(dim=1)==y).float().mean()

            # P5.4. flush out the previously computed gradient 
            # write your code here (one-liner).

            # P5.5. backward the computed loss. 
            # write your code here (one-liner).

            # P5.6. update the network weights. 
            # write your code here (one-liner).

            if global_step % args.log_iter == 0 and writer is not None:
                # P5.7. Log `loss` with a tag name 'train_loss' using `writer`. Use `global_step` as a timestamp for the log. 
                # writer.writer_your_code_here (one-liner).
                # P5.8. Log `accuracy` with a tag name 'train_accuracy' using `writer`. Use `global_step` as a timestamp for the log. 
                # writer.writer_your_code_here (one-liner).

        # print train loss, acc, time spent 
        t = time.time()-_start_time
        print(f'Epoch {epoch}/{args.epoch} || train loss={loss:.4f} train acc={accuracy*100:.3f}% time={t:.3f} secs')

        # start time for test
        _start_time = time.time() 

        # Here starts the test loop.
        net.eval()
        with torch.no_grad():
            test_loss = 0.
            test_accuracy = 0.
            test_num_data = 0.
            for x, y in test_dataloader:
                # P5.9. Send `x` and `y` to either cpu or gpu using `device` variable.
                # x = write your code here (one-liner).
                # y = write your code here (one-liner).

                # P5.10. Feed `x` into the network, get an output, and keep it in a variable called `logit`.
                # logit = write your code here (one-liner). 

                # P5.11. Compute loss using `logit` and `y`, and keep it in a variable called `loss`
                # loss = write your code yere (one-liner). 
                accuracy = (logit.argmax(dim=1) == y).float().mean()

                test_loss += loss.item()*x.shape[0]
                test_accuracy += accuracy.item()*x.shape[0]
                test_num_data += x.shape[0]

            test_loss /= test_num_data
            test_accuracy /= test_num_data

            if writer is not None: 
                # P5.12. Log `test_loss` with a tag name 'test_loss' using `writer`. Use `global_step` as a timestamp for the log.
                # writer.write_your_code_here (one-liner).
                # P5.13. Log `test_accuracy` with a tag name 'test_accuracy' using `writer`. Use `global_step` as a timestamp for the log.
                # writer.write_your_code_here (one-liner).
                writer.flush()

            # P5.14. Whenever `test_accuracy` is greater than `best_accuracy`, save network weights with the filename 'best.pt' in the directory specified by `ckpt_dir`.
            #     Here, just save the network weights (i.e, you don't need to save optimizer)
            #     Also, don't forget to update the `best_accuracy` properly.
            # write your code here. 

        # P5.15. Save the checkpoint in the directory specified by `ckpt_dir` directory. 
        #    Note that the checkpoint must include network weights, optmizer states, current epoch, best acuuracy so far. 
        #    To see how those parameters are loaded, see the above cell that loads the checkpoint and resumes the training. 
        #    Hint) Write something like, torch.save(dict(epoch=, net=, optimizer=, best_accuracy=), checkpoint_filename)
        #    Also, use `epoch` to specify the timestamp in the checkpoint filename.
        #    E.g) if `epoch=10`, the filename can be `10.pt`
        # write your code here (one-liner).

        # print test loss, acc, time spent
        t = time.time()-_start_time
        print(f'Epoch {epoch}/{args.epoch} || test loss={loss:.4f} test acc={test_accuracy*100:.3f}% time={t:.3f} secs')

    # Print final accuracy with total time spent for training
    total_t = time.time()-start_time 
    print(f'Final best accuracy : {best_accuracy*100:.3f}% total time={total_t:.3f} secs')

In [ ]:
# Training
train(args)

---
# 6. Discussions (50pt)

Train and test at least 3 models with different configurations (for example, test with `ResBlockPlain` block instead of `ResBlockBottleneck` or you may stack more layers, etc) and hyper-parameters and discuss the results. Simply reporting the results (e.g. classification accuracy) is not considered as a discussion. You should explain which components lead to differences and analyze the reason for those differences. 

For the experiments, you can change the configurations and hyper-parameters by modifying the values written in the cell defining configurations and hyper-parameters. **Also, don't forget to change `args.name` before you run the new experiment!** 

Then, run the experiments below and leave the logs including test accuracy as a proof that you actually conducted the experiments. Based on the experimental results, write your discussions. 

* Experiment #1 

In [ ]:
# Run your Experiment here
# You may add cells if you want

* Experiment #2

In [ ]:
# Run your Experiment here
# You may add cells if you want

* Experiment #3

In [ ]:
# Run your Experiment here
# You may add cells if you want

* Your discussions.

Write disccusions here...